In [1]:
import numpy as np
import pandas as pd
import sys,os

def load_files(batch):
    images = []
    labels = []
    for i in batch:
        print('Loading File: ' + i)
        x = np.load(i,encoding = 'latin1').item()
        keys = x.keys()
        for key in keys:
            images.append(x[key][0])
            labels.append(x[key][1])
    return np.array(images),np.array(labels)

def get_feature(labels,feature):
    feature_values = []
    for i in labels:
        feature_values.append(i[feature])
    feature_values = np.array(feature_values)
    return feature_values

def get_cos_values(zenith,azimuth):
    cos1 = []
    cos2 = []
    cos3 = []
    for i,j in zip(zenith,azimuth):
        cos1.append(np.sin(i) * np.cos(j))
        cos2.append(np.sin(i) * np.sin(j))
        cos3.append(np.cos(i))
    return np.array(cos1),np.array(cos2),np.array(cos3)

In [3]:
file_path = '/fs/scratch/PAS1495/amedina/'
y = os.listdir(file_path+'processed_new')

file_names = []

for i in y:
    file_names.append(file_path+'processed_new/'+i)

file_names_batched = list(np.array_split(file_names,400))

FileNotFoundError: [WinError 3] The system cannot find the path specified: '/fs/scratch/PAS1495/amedina/processed_new'

In [ ]:
images,labels = load_files(file_names_batched[0])

zenith_values = get_feature(labels,1)
azimuth_values = get_feature(labels,2)

cos1,cos2,cos3 = get_cos_values(zenith_values,azimuth_values)

In [ ]:
import keras
from keras.models import Sequential,load_model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D , SeparableConv2D , GlobalAveragePooling2D, BatchNormalization
from keras import backend as K
from sklearn.model_selection import train_test_split,KFold
from keras.layers import LeakyReLU, PReLU
from keras import regularizers

In [ ]:
cos_values = np.array(zip(cos1,cos2,cos3))

In [ ]:
def get_data(images,true_values):
    x_train, x_test , y_train , y_test = train_test_split(images,true_values,test_size = 0.2 , random_state=42)
    x_train = x_train.reshape([x_train.shape[0],x_train.shape[1],x_train.shape[2],1])
    x_test = x_test.reshape([x_test.shape[0],x_test.shape[1],x_test.shape[2],1])
    return x_train,x_test,y_train,y_test

x_train,x_test,y_train,y_test = get_data(images,cos_values)

In [ ]:
import tensorflow as tf

sess = tf.Session()

def loss_space_angle(y_true,y_pred):
    subtraction = tf.math.subtract(y_true,y_pred)
    y = tf.matrix_diag_part(K.dot(subtraction,K.transpose(subtraction)))
    loss = tf.math.reduce_mean(y)
    return loss

y = loss_space_angle(y_train[0:166],y_test)
print(y.eval(session=sess))

In [ ]:
batch_size = 128
epochs=20

img_rows, img_cols = 300,342
input_shape = (img_rows, img_cols)

kernel = 3
kernel2 = 2

model = Sequential()
model.add(SeparableConv2D(16,kernel,
                          input_shape = (img_rows,img_cols,1)))
model.add(LeakyReLU(alpha = 0.01))
model.add(SeparableConv2D(16,kernel))
model.add(LeakyReLU(alpha = 0.01))
model.add(SeparableConv2D(16,kernel))
model.add(LeakyReLU(alpha = 0.01))
model.add(MaxPooling2D(kernel2))

model.add(Dropout(0.1))

model.add(SeparableConv2D(16,kernel))
model.add(LeakyReLU(alpha = 0.01))
model.add(SeparableConv2D(16,kernel))
model.add(LeakyReLU(alpha = 0.01))
model.add(SeparableConv2D(16,kernel))
model.add(LeakyReLU(alpha = 0.01))
model.add(MaxPooling2D(kernel2))

model.add(Dropout(0.1))

model.add(SeparableConv2D(16,kernel))
model.add(LeakyReLU(alpha = 0.01))
model.add(SeparableConv2D(16,kernel))
model.add(LeakyReLU(alpha = 0.01))
model.add(SeparableConv2D(16,kernel))
model.add(LeakyReLU(alpha = 0.01))
model.add(MaxPooling2D(kernel2))

model.add(SeparableConv2D(16,kernel))
model.add(LeakyReLU(alpha = 0.01))
model.add(SeparableConv2D(16,kernel))
model.add(LeakyReLU(alpha = 0.01))
model.add(SeparableConv2D(16,kernel))
model.add(LeakyReLU(alpha = 0.01))
model.add(MaxPooling2D(kernel2))

model.add(Dropout(0.1))

model.add(Flatten())

model.add(Dense(32,activation='linear'))
model.add(LeakyReLU(alpha = 0.01))
model.add(Dense(3))

opt = keras.optimizers.SGD(lr=0.01, decay=1e-6, momentum=.09, nesterov=True)
model.compile(optimizer=opt , loss = loss_space_angle)

In [ ]:
history=model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))

In [ ]:
import matplotlib.pyplot as plt
history_dict = history.history
print(history_dict.keys())
loss_values = history_dict['loss']
val_loss_values = history_dict['val_loss']

epochs = range(1, len(loss_values)+1)

plt.plot(epochs,loss_values,'bo',label='Training loss')
plt.plot(epochs,val_loss_values,'b',label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()